In [1]:
import os
os.chdir('/kaggle/input/myfunctions')
from functions import *
from myutils import *
import pandas as pd
import numpy as np
import pickle
from functions import *
os.chdir('/kaggle/input/mydatasets2')
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tqdm import tqdm

In [2]:
time_steps = 40 # Time steps to consider
df_path = '/kaggle/input/mydatasets2/training_set1.pkl'

In [3]:
data, labels = preprocess_data_via_close_values(df_path, time_steps=time_steps, shuffle=True, split=False)
print(data.shape)
print(labels.shape)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (2161, 40)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 2085072 2116546 120382
Combined array shape -  (4322000, 40)
Combined labels shape -  (4322000,)
Shuffled data shape -  (4322000, 40)
(4322000, 40)
(4322000,)


In [8]:
test_size=0.2
random_state=42
x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=test_size, random_state=random_state)
num_bagging_splits = 100

In [4]:
models_dict = {}
params = {'objective':'multi:softmax', 'eval_metric':'merror', 'num_class':'3', 'eta':'0.1', 'max_depth':'5'}
num_boost_rounds = 100
scaling = 2.5
for i in tqdm(range(num_bagging_splits)):
    indices = np.arange(0, len(x_train))
    np.random.shuffle(indices)
    slice_val = int(len(indices)//scaling)
    indices_selected = indices[:slice_val]
    x_tr = x_train[indices_selected]
    y_tr = y_train[indices_selected]
    xg_train = xgb.DMatrix(x_tr, label=y_tr)
    models_dict[i] = xgb.train(params, xg_train, num_boost_round=num_boost_rounds)

100%|██████████| 100/100 [1:23:57<00:00, 50.38s/it]


In [5]:
def xg_multi_predict(x, y, models_dict):
    xg_valid = xgb.DMatrix(x, label=y)
    y_pred = np.zeros_like(y, dtype=np.float32)
    # y_pred = models_dict[0].predict(xg_valid)
    for xg_model in models_dict.values():
        y_pred += xg_model.predict(xg_valid)
    y_pred = y_pred/len(models_dict)
    return y_pred

In [6]:
models_dict = load_file_from_pickle('/kaggle/input/models-xg-bagging-100-40/models_xg_bagging_100_40.pkl')

In [9]:
metric_keys = ("accuracy", "precision", "specificity", "recall", "f1score")
y_train_pred = xg_multi_predict(x = x_train, y = y_train, models_dict = models_dict)
y_train_pred = np.round(y_train_pred)

In [10]:
train_metrics = dict(zip(metric_keys, metric_calculations(y_train_pred, y_train, set_="Train set")))

For Train set
  Accuracy: 38.16627140212864,
  Precision: 50.5507425967069,
  Recall: 31.83265703484129,
  Specificity: 48.61738754500165,
  F1score: 39.06525972791475


In [11]:
y_pred = xg_multi_predict(x = x_valid, y = y_valid, models_dict = models_dict)
y_pred = np.round(y_pred)

In [12]:
valid_metrics = dict(zip(metric_keys, metric_calculations(y_pred, y_valid, set_="Validation set")))

For Validation set
  Accuracy: 38.13373438223045,
  Precision: 50.47020555478735,
  Recall: 31.784451573695637,
  Specificity: 48.596994257828435,
  F1score: 39.00491134173399


In [13]:
df_path_test = "/kaggle/input/mydatasets2/training_set11.pkl"
x_test, y_test = preprocess_data_via_close_values(df_path_test, time_steps=time_steps, shuffle=True, split=False)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (848, 40)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 1018328 1004558 52170
Combined array shape -  (2075056, 40)
Combined labels shape -  (2075056,)
Shuffled data shape -  (2075056, 40)


In [14]:
y_pred_test = xg_multi_predict(x = x_test, y = y_test, models_dict = models_dict)
y_pred_test = np.round(y_pred_test)

In [15]:
test_metrics = dict(zip(metric_keys, metric_calculations(y_pred_test, y_test, set_="Test set")))

For Test set
  Accuracy: 33.49977060860044,
  Precision: 49.70781541688521,
  Recall: 28.66786593565052,
  Specificity: 42.99602443013036,
  F1score: 36.36375374947149


In [16]:
output_keys = ["models", "train_metrics", "validation_metrics", "test metrics"]
output_values = [models_dict, train_metrics, valid_metrics, test_metrics]

In [17]:
output_file_path = f"/kaggle/working/models_xg_bagging_{num_bagging_splits}_{time_steps}.pkl"
write_file_to_pickle(output_file_path, output_values)